In [3]:
# get price data (previous close price)
import requests
import json

with open('config.json', 'r') as file:
    config = json.load(file)

api_key = config['api_key']
url = 'https://api.polygon.io/v2/aggs/ticker/ETH/prev'

params = {
    'adjusted': 'true',
    'apiKey': api_key
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print(data['results'][0]['c'])
else:
    print(f"Error: {response.status_code}")


In [7]:
from qiskit import QuantumCircuit


sigma = 0.3
n_coin = 1
n_price = 4 # precision for price
idx_coin = 4
circuit = QuantumCircuit(n_price + n_coin, n_price)

circuit.h(idx_coin)

n = 50
for i in range(n):

    circuit.h(idx_coin)
    for i in range(n_price):
        circuit.cx(idx_coin, i)

circuit.measure(range(n_price), range(n_price))

circuit.draw()


┌───┐                    ┌───┐                    ┌───┐     »
q_0: ──────────┤ X ├────────────────────┤ X ├────────────────────┤ X ├─────»
               └─┬─┘┌───┐               └─┬─┘┌───┐               └─┬─┘┌───┐»
q_1: ────────────┼──┤ X ├─────────────────┼──┤ X ├─────────────────┼──┤ X ├»
                 │  └─┬─┘┌───┐            │  └─┬─┘┌───┐            │  └─┬─┘»
q_2: ────────────┼────┼──┤ X ├────────────┼────┼──┤ X ├────────────┼────┼──»
                 │    │  └─┬─┘┌───┐       │    │  └─┬─┘┌───┐       │    │  »
q_3: ────────────┼────┼────┼──┤ X ├───────┼────┼────┼──┤ X ├───────┼────┼──»
     ┌───┐┌───┐  │    │    │  └─┬─┘┌───┐  │    │    │  └─┬─┘┌───┐  │    │  »
q_4: ┤ H ├┤ H ├──■────■────■────■──┤ H ├──■────■────■────■──┤ H ├──■────■──»
     └───┘└───┘                    └───┘                    └───┘          »
c: 4/══════════════════════════════════════════════════════════════════════»
                                                                           »
«                    ┌───┐                    ┌───┐                    ┌───┐»
«q_0: ───────────────┤ X ├────────────────────┤ X ├────────────────────┤ X ├»
«                    └─┬─┘┌───┐               └─┬─┘┌───┐               └─┬─┘»
«q_1: ─────────────────┼──┤ X ├─────────────────┼──┤ X ├─────────────────┼──»
«     ┌───┐            │  └─┬─┘┌───┐            │  └─┬─┘┌───┐            │  »
«q_2: ┤ X ├────────────┼────┼──┤ X ├────────────┼────┼──┤ X ├────────────┼──»
«     └─┬─┘┌───┐       │    │  └─┬─┘┌───┐       │    │  └─┬─┘┌───┐       │  »
«q_3: ──┼──┤ X ├───────┼────┼────┼──┤ X ├───────┼────┼────┼──┤ X ├───────┼──»
«       │  └─┬─┘┌───┐  │    │    │  └─┬─┘┌───┐  │    │    │  └─┬─┘┌───┐  │  »
«q_4: ──■────■──┤ H ├──■────■────■────■──┤ H ├──■────■────■────■──┤ H ├──■──»
«               └───┘                    └───┘                    └───┘     »
«c: 4/══════════════════════════════════════════════════════════════════════»
«                                                                           »
«                         ┌───┐                    ┌───┐                    »
«q_0: ────────────────────┤ X ├────────────────────┤ X ├────────────────────»
«     ┌───┐               └─┬─┘┌───┐               └─┬─┘┌───┐               »
«q_1: ┤ X ├─────────────────┼──┤ X ├─────────────────┼──┤ X ├───────────────»
«     └─┬─┘┌───┐            │  └─┬─┘┌───┐            │  └─┬─┘┌───┐          »
«q_2: ──┼──┤ X ├────────────┼────┼──┤ X ├────────────┼────┼──┤ X ├──────────»
«       │  └─┬─┘┌───┐       │    │  └─┬─┘┌───┐       │    │  └─┬─┘┌───┐     »
«q_3: ──┼────┼──┤ X ├───────┼────┼────┼──┤ X ├───────┼────┼────┼──┤ X ├─────»
«       │    │  └─┬─┘┌───┐  │    │    │  └─┬─┘┌───┐  │    │    │  └─┬─┘┌───┐»
«q_4: ──■────■────■──┤ H ├──■────■────■────■──┤ H ├──■────■────■────■──┤ H ├»
«                    └───┘                    └───┘                    └───┘»
«c: 4/══════════════════════════════════════════════════════════════════════»
«                                                                           »
«     ┌───┐                    ┌───┐                    ┌───┐               »
«q_0: ┤ X ├────────────────────┤ X ├────────────────────┤ X ├───────────────»
«     └─┬─┘┌───┐               └─┬─┘┌───┐               └─┬─┘┌───┐          »
«q_1: ──┼──┤ X ├─────────────────┼──┤ X ├─────────────────┼──┤ X ├──────────»
«       │  └─┬─┘┌───┐            │  └─┬─┘┌───┐            │  └─┬─┘┌───┐     »
«q_2: ──┼────┼──┤ X ├────────────┼────┼──┤ X ├────────────┼────┼──┤ X ├─────»
«       │    │  └─┬─┘┌───┐       │    │  └─┬─┘┌───┐       │    │  └─┬─┘┌───┐»
«q_3: ──┼────┼────┼──┤ X ├───────┼────┼────┼──┤ X ├───────┼────┼────┼──┤ X ├»
«       │    │    │  └─┬─┘┌───┐  │    │    │  └─┬─┘┌───┐  │    │    │  └─┬─┘»
«q_4: ──■────■────■────■──┤ H ├──■────■────■────■──┤ H ├──■────■────■────■──»
«                         └───┘                    └───┘                    »
«c: 4/══════════════════════════════════════════════════════════════════════»
«                                                 

In [ ]:
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

simulator = AerSimulator()

compiled_circuit = transpile(circuit, simulator)

res = simulator.run(compiled_circuit).result()
counts = res.get_counts()

plot_histogram(counts)

In [ ]:
from qiskit import QuantumCircuit

# 1) initialize price register, coin register (precision: 3-4 bits)

# 2) repeat the walk for 100-200 steps:

# 